# Примечание: сейчас этот файл не работает, но в конце файла видно, что цикл парсинга у него отработал нормально.

In [47]:
#!pip install fake_useragent

In [3]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook

## Описание задачи

Парсинг сайта https://topliba.com/ с целью сбора данных (рейтинг, название фильма, описание, жанр, год выпуска) по жанру Фэнтези. Дело в том, что большинство современных книжный сайтов не предоставляют возможности делать сквозной  поиск по всем страницам, а только в рамках делений по жанрам. Наиболее представленный жанр н данном сайте - фэнтези (783 страницы поиска) по нему и будем делать парсинг. На данном сайте список фильмов подгружается динамически по страницам. Обратимся к странице Фэнтези, и запишем запрос к данной странице.

In [4]:
page_link = 'https://topliba.com/genres/12090?p=1'

In [5]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [6]:
html = response.content

### Получение ссылок с фильмами

С помощью библиотеки BeautifulSoup распарсим страницу и найдем ссылки к фильмам. 
Ссылки к фильмам находятся в теге div с классом discovery-grids-container.

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html lang="ru">\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n        <title>\xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8: \xd1\x81\xd0\xba\xd0\xb0\xd1\x87\xd0\xb0\xd1\x82\xd1\x8c \xd0\xba\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb2 fb2, \xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbe\xd0\xbd\xd0\xbb\xd0\xb0\xd0\xb9\xd0\xbd \xe2\x80\xa2 \xd0\xa1\xd1\x82\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x86\xd0\xb0 1</title>\n        <meta name="description" content="\xd0\x9a\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb6\xd0\xb0\xd0\xbd\xd1\x80\xd0\xb0 \xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8 \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd1\x83\xd0\xbf\xd0\xbd\xd1\x8b\xd0\xb5 \xd0\xb2 \xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd0\xb0\xd1\x82\xd0\xb5 fb2, \xd0\xb0 \xd1\x82\xd0\xb0\xd0\xba\xd0\xb6\xd0\xb5 \xd0\xb4\xd0\xbb\xd1\x8f \xd1\x

In [9]:
# Cписок ссылок на каждую книгу со страницы
book_links = [link.attrs['href'] for link in 
                  soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])]
book_links

['https://topliba.com/books/898032',
 'https://topliba.com/books/897776',
 'https://topliba.com/books/898031',
 'https://topliba.com/books/897772',
 'https://topliba.com/books/897639',
 'https://topliba.com/books/897636',
 'https://topliba.com/books/897633',
 'https://topliba.com/books/897397',
 'https://topliba.com/books/897225',
 'https://topliba.com/books/897388',
 'https://topliba.com/books/897224',
 'https://topliba.com/books/897276',
 'https://topliba.com/books/897272',
 'https://topliba.com/books/897246',
 'https://topliba.com/books/897229',
 'https://topliba.com/books/897209',
 'https://topliba.com/books/896911',
 'https://topliba.com/books/896731',
 'https://topliba.com/books/897016',
 'https://topliba.com/books/896710',
 'https://topliba.com/books/896566',
 'https://topliba.com/books/896488',
 'https://topliba.com/books/896219',
 'https://topliba.com/books/896190']

In [10]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на книги, полученный с текущей страницы
        
        Параметры:
            page_number: int/string - номер страницы для парсинга            
    """
    # составляем ссылку на страницу поиска
    page_link = 'https://topliba.com/genres/12090?p={}'.format(page_number)
    
    # запрашиваем данные о ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        print('Сервер ничего не вернул')
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # ищем ссылки на книги и формируем полные ссылки
    book_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])
    book_links = [link.attrs['href'] for link in book_links]
    
    return book_links

In [11]:
book_links = getPageLinks(3)
book_links

['https://topliba.com/books/891793',
 'https://topliba.com/books/891792',
 'https://topliba.com/books/891095',
 'https://topliba.com/books/891044',
 'https://topliba.com/books/890657',
 'https://topliba.com/books/890813',
 'https://topliba.com/books/889932',
 'https://topliba.com/books/889824',
 'https://topliba.com/books/889772',
 'https://topliba.com/books/889771',
 'https://topliba.com/books/883986',
 'https://topliba.com/books/887325',
 'https://topliba.com/books/883928',
 'https://topliba.com/books/889619',
 'https://topliba.com/books/889141',
 'https://topliba.com/books/889278',
 'https://topliba.com/books/889169',
 'https://topliba.com/books/888662',
 'https://topliba.com/books/888765',
 'https://topliba.com/books/888636',
 'https://topliba.com/books/888704',
 'https://topliba.com/books/888672',
 'https://topliba.com/books/888635',
 'https://topliba.com/books/888604']

### Получим информации о книге
Обратимся к странице с фильмом Сердце Дракона. Том 2 и достанем следующую информацию:
- Название книги.
- Автор книги
- Оценка книги.
- Количество читателей
- Количество комментариев/отзывов
- Аннотация.
- Жанр книги.
- Год выпуска.

In [12]:
book_page = 'https://topliba.com/books/736796'
response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(response.content, 'html.parser')

### Получим название книги

In [13]:
book_title = soup.find(lambda tag: tag.name == 'h1') # and tag.get('itemprop') == ['name'])
book_title = book_title.text
book_title

'Щит императора'

### Получим автора книги

In [14]:
book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
book_author = book_author.text.replace("\n", "")
book_author

'Александра Лисина'

### Получим рейтинг книги 

Рейтингикниг хранятся в аттрибутах тега div class="col-lg-12". Посмотрим на структура данного тега.

In [15]:
scoreboard = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['col-lg-12'])
scoreboard

<div class="col-lg-12">
<rating _avgrating="4.75" _rating="0" _ratingsnumber="251" _showavgrating="true" _showavgratingtext="true" _url="/books/736796/ratings" _user="" class="pull-left"></rating>
<meta content="1" itemprop="worstRating"/>
<meta content="5" itemprop="bestRating"/>
<meta content="251" itemprop="ratingCount"/>
<meta content="4.75" itemprop="ratingValue"/>
</div>

In [16]:
def getRating(soup, rate):
    try:
        obj = soup.find('rating', attrs={'class':'pull-left'})
        obj = obj.get(rate)
    except:
        obj=None
    
    return obj

In [17]:
getRating(soup, '_avgrating')

'4.75'

### Получим количество читателей книги и количества комментариев (отзывов)

Сведения о читателях хранятся в book-readers

In [18]:
book_readers = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['book-readers'])
book_readers

<div class="book-readers">
<div class="col-lg-8 col-sm-12 well-sm">
<div class="col-sm-2 col-xs-4 book-readers-status">
<span class="book-readers-status-count">490</span>
<span>читателей</span>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/impressions">
<span class="book-readers-status-count">88</span>
<span>отзывов</span>
</a>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/quotes">
<span class="book-readers-status-count">20</span>
<span>цитат</span>
</a>
</div>
</div>
</div>

In [19]:
# Находим все span элементы с классом 'book-readers-status-count'
readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

book_readership_count = readers_count_elements[0].string  
book_comment_count = readers_count_elements[1].string  

print("Количество читателей:", book_readership_count)
print("Количество отзывов:", book_comment_count)

Количество читателей: 490
Количество отзывов: 88


### Получим аннотацию к книге
Аннотация находится в параграфе с классом fsBigX. 
Также удалим лишние пробелы в начале и конце строки и заменим неразрывный пробел обычным пробелом.

In [20]:
book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
book_announs = book_announs.strip()
book_announs

'После смерти великого императора Орриана на престол вступает его единственный сын, которому в наследство достается не только страна, но и телохранитель. Карриану не нужен такой довесок к короне, однако нарушить волю отца он не рискует. Мару тоже не нравится это решение, но приказ хозяина для него превыше собственных предпочтений. Для двух врагов подобное соседство — худшее, что только можно измыслить. Тень, не желающая оберегать нового императора, и император, ненавидящий свою тень… Казалось бы, у этих двоих нет и не может быть ничего общего. Но жизнь все расставляет по своим местам.'

### Получение данных о жанре

In [86]:
# Находим все теги div с классом 'col-lg-12'
search_elements = soup.find_all('div', class_='col-lg-12')
search_elements

[<div class="col-lg-12">
 <rating _avgrating="4.75" _rating="0" _ratingsnumber="251" _showavgrating="true" _showavgratingtext="true" _url="/books/736796/ratings" _user="" class="pull-left"></rating>
 <meta content="1" itemprop="worstRating"/>
 <meta content="5" itemprop="bestRating"/>
 <meta content="251" itemprop="ratingCount"/>
 <meta content="4.75" itemprop="ratingValue"/>
 </div>,
 <div class="col-lg-12">
 <a href="https://topliba.com/genres/12090" title="Книги жанра Фэнтези">Фэнтези</a>, <a href="https://topliba.com/genres/12147" title="Книги жанра Попаданцы">Попаданцы</a>
 </div>,
 <div class="col-lg-12">
         Год выхода: <a href="https://topliba.com/years/2020" title="Книги 2020 года">2020</a>
 </div>,
 <div class="col-lg-12">
             Серия: <a href="https://topliba.com/series/267" title='Книги серии "Магия фэнтези"'>Магия фэнтези</a>, <a href="https://topliba.com/series/370269" title='Книги серии "Слуга императора"'>Слуга императора</a> (#2)
         </div>,
 <div clas

div class="col-lg-12"><a href="https://topliba.com/genres/12090" title="Книги жанра Фэнтези">Фэнтези</a>, <a href="https://topliba.com/genres/12147" title="Книги жанра Попаданцы">Попаданцы</a></div>

Запишем итоговую функцию для сбора данных по книге.

In [105]:
def getBookData(film_page):
    book_data = pd.DataFrame(columns=['Author', 'Description', 'Readers', 'Comments'], index=[0])
    
    response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(response.content, 'html.parser')
           
    book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
    book_author = book_author.text.replace("\n", "")
    book_data['Author'] = book_author
    
    book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
    book_announs = book_announs.strip()
    book_data['Description']= book_announs
      
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
    book_data['Readers'] = book_readership_count
    
    book_comment_count = readers_count_elements[1].string  
    book_data['Comments'] = book_comment_count
    
    return book_data

In [106]:
getBookData('https://topliba.com/books/736796')

,Author,Description,Readers,Comments
0,Александра Лисина,После смерти великого императора Орриана на пр...,490,88


# Парсинг

## Получение информации по всем книгам
Соберем информацию по всем книгам. Для начала создадим пустой датафрейм, в который мы будем добавлять информацию по каждой книге.

In [122]:
final_df = pd.DataFrame(columns=['Author', 'Description', 'Readers', 'Comments'])
final_df.head()

,Author,Description,Readers,Comments


In [123]:
book_links = getPageLinks(1000)

Цикл для сбора информации по всем фильмам.

In [139]:
book_data_list = []
for book_link in tqdm(book_links):
    try: 
        book_data = getBookData(book_link)
        book_data_list.append(book_data)
        time.sleep(3)
    except Exception as e:
        print("Ошибка при парсинге", book_link)
        print(e)

final_df = pd.DataFrame(book_data_list)

  0%|          | 0/24 [00:00<?, ?it/s]

In [146]:
final_df.head()

,Author,Description,Readers,Comments
0,Надежда Соколова,"Никто не спрашивал меня, когда отправил в друг...",244,21
1,Крепкая Элья,«В борьбе двух хищников побеждают только падал...,39,3
2,Наталья Якобсон,"Младший из трех принцев красив, умен, любим в ...",22,0
3,Рианнон Илларионова,Главная героиня отправляется учиться в школу з...,5,0
4,Сумеречный Эльф,"Капитан Сварт, мстительный пират, давно мечтае...",6,0


In [145]:
from tqdm.notebook import tqdm

for book_link in tqdm(book_links):
    try: 
        book_data = getData(book_link)
        final_df = pd.concat((final_df, book_data), axis=0)
        time.sleep(3)
    except:
        continue

  0%|          | 0/24 [00:00<?, ?it/s]

In [141]:
final_df.head(30)

,Author,Description,Readers,Comments
0,Надежда Соколова,"Никто не спрашивал меня, когда отправил в друг...",244,21
1,Крепкая Элья,«В борьбе двух хищников побеждают только падал...,39,3
2,Наталья Якобсон,"Младший из трех принцев красив, умен, любим в ...",22,0
3,Рианнон Илларионова,Главная героиня отправляется учиться в школу з...,5,0
4,Сумеречный Эльф,"Капитан Сварт, мстительный пират, давно мечтае...",6,0
5,Axiom,Между империей Бима и царством Охвот разжигает...,9,0
6,Алиса Элер,"Говорят, однажды посмотрев в глаза Ши, никогда...",9,0
7,Алиса Элер,"""Ветер, холодный и злой, рвет плащ, срывает с ...",10,0
8,Андрей Пучков,Ты живёшь в своей квартире спокойной и размере...,9,0
9,Иван Шмелев,Аллен становится признанным Специалистом. Если...,7,0
